In [11]:
import pandas as pd
import torch
import albumentations
import numpy as np
from PIL import Image, ImageFile
import string
import os   
from config import  DATA_DIR, BATCH_SIZE, IMAGE_WIDTH, IMAGE_HEIGHT, EPOCHS, LEARNING_RATE, NUM_WORKERS, DEVICE


In [12]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

class dataset:
    def __init__(self, image_path, targets, resize=None):
        self.image_path = image_path
        self.targets = targets
        self.resize = resize
        self.aug = albumentations.Compose([albumentations.Normalize(always_apply=True)])
        
        # mapping huruf ke angka
        alphabet = string.ascii_lowercase   # 'abcdefghijklmnopqrstuvwxyz'
        self.char2idx = {ch: idx+1 for idx, ch in enumerate(alphabet)}  # mulai dari 1
        self.char2idx["<pad>"] = 0  # buat padding jika perlu

    def __len__(self):
        return len(self.image_path)
    
    def text_to_seq(self, text):
        """Ubah string label jadi list angka huruf per huruf"""
        return [self.char2idx[c] for c in text.lower() if c in self.char2idx]

    def __getitem__(self, item):
        # ambil gambar
        image = Image.open(self.image_path[item]).convert("RGB")
        targets = self.targets[item]   # string kata, misal "kucing"

        if self.resize is not None:
            image = image.resize((self.resize[0], self.resize[1]), resample=Image.BILINEAR)

        image = np.array(image)
        augmented = self.aug(image=image)
        image = augmented['image']
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)

        # ubah kata jadi list angka
        target_seq = self.text_to_seq(targets)

        return {
            "images": torch.tensor(image, dtype=torch.float),
            "targets": torch.tensor(target_seq, dtype=torch.long),   # urutan huruf dalam bentuk tensor
            "target_length": torch.tensor(len(target_seq), dtype=torch.long)  # panjang kata
        }


In [ ]:
import pandas as pd

df = pd.read_csv("labels.csv")
df["filepath"] = df["filename"].apply(lambda x: f"{DATA_DIR}/{x}")
df.to_csv("labels_with_path.csv", index=False)


In [ ]:
df = pd.read_csv("labels_with_path.csv")
image_paths = df["filepath"].tolist()
targets = df["label"].tolist()




In [15]:
df

,filename,label,filepath
0,img_0.png,xjdv,dataset_pil/img_0.png
1,img_1.png,nwxff,dataset_pil/img_1.png
2,img_2.png,wugb,dataset_pil/img_2.png
3,img_3.png,omapsdso,dataset_pil/img_3.png
4,img_4.png,ehjcxhg,dataset_pil/img_4.png
...,...,...,...
79995,img_79995.png,mrokrhpft,dataset_pil/img_79995.png
79996,img_79996.png,ntuvxev,dataset_pil/img_79996.png
79997,img_79997.png,uybeusnx,dataset_pil/img_79997.png
79998,img_79998.png,rqtju,dataset_pil/img_79998.png


In [ ]:
dataset = dataset(image_path=image_paths, targets=targets)

In [17]:
sample = dataset[0]
print(sample["images"].shape)
print(sample["targets"])
print(sample["target_length"])

torch.Size([3, 80, 300])
tensor([24, 10,  4, 22])
tensor(4)


In [18]:
len (dataset)

80000